In [30]:
desc_folder = './results'
colName_mid = 'Compound Name'

desc_rdkit = True
desc_fps = True
desc_cx = True
desc_custom = True
model_type = 'regression'    # 'classification'

MissingValueFilter = True
VarianceFilter = True
L2Filter = True
FeatureImportanceFilter = True

##
descType_list = []
if desc_custom:
    descType_list.append('custom')
if desc_rdkit:
    descType_list.append('rdkit')
if desc_fps:
    descType_list.append('fingerprints')
if desc_cx:
    descType_list.append('chemaxon')
print(f"\tSelected descriptors types: {descType_list}")

	Selected descriptors types: ['custom', 'rdkit', 'fingerprints', 'chemaxon']


In [31]:
import pandas as pd

## load the expt outcome
dataTable_y = pd.read_csv(f'{desc_folder}/outcome_expt.csv')
print(dataTable_y.shape)
dataTable_y.head(3)

(38, 2)


,Compound Name,ADME MDCK(WT) Permeability;Mean;A to B Papp (10^-6 cm/s);(Num)
0,KT-0001674,0.298013
1,KT-0001675,0.767547
2,KT-0001676,0.186651


In [32]:
## load all descriptor tables and merge together
dataTable_merged = dataTable_y

for descType in descType_list:
    dataTable = pd.read_csv(f'{desc_folder}/descriptors_{descType}_processed.csv') 
    print(f"\tThere are total <{dataTable.shape[1]-1}> {descType} descriptors for <{dataTable.shape[0]}> molecules")
    dataTable_merged = dataTable_merged.merge(right=dataTable, on='Compound Name', how='left')
print(f"\tThe merged data table has <{dataTable_merged.shape[0]}> molecules and <{dataTable_merged.shape[1]-1}> descriptors")
dataTable_merged.head(5)

	There are total <2> custom descriptors for <38> molecules
	There are total <140> rdkit descriptors for <38> molecules
	There are total <2048> fingerprints descriptors for <38> molecules
	There are total <33> chemaxon descriptors for <38> molecules
	The merged data table has <38> molecules and <2224> descriptors


,Compound Name,ADME MDCK(WT) Permeability;Mean;A to B Papp (10^-6 cm/s);(Num),custDesc_ADME MDCK(WT) Permeability;Mean;A to B Recovery (%),custDesc_ADME MDCK(WT) Permeability;Mean;B to A Papp (10^-6 cm/s);(Num),rd_MaxAbsEStateIndex,rd_MaxEStateIndex,rd_MinAbsEStateIndex,rd_MinEStateIndex,rd_qed,rd_SPS,...,cx_topology-analyser_chiralCenterCount,cx_topology-analyser_fsp3,cx_topology-analyser_fusedAliphaticRingCount,cx_topology-analyser_fusedAromaticRingCount,cx_topology-analyser_fusedRingCount,cx_topology-analyser_heteroRingCount,cx_topology-analyser_heteroAromaticRingCount,cx_topology-analyser_largestRingSize,cx_topology-analyser_largestRingSystemSize,cx_topology-analyser_rotatableBondCount
0,KT-0001674,0.298013,-0.012508,-0.449259,1.820147,1.820147,0.753577,1.248948,0.347800,-0.560154,...,-0.766791,0.171082,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.397809
1,KT-0001675,0.767547,0.034798,1.291708,1.808079,1.808079,2.133891,1.271090,0.299537,-0.447405,...,-0.766791,0.737510,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.178726
2,KT-0001676,0.186651,-2.419224,-0.765732,-1.339989,-1.339989,-0.372173,-0.569409,-0.658017,-1.698210,...,-1.386750,-0.142262,-0.342997,-0.190693,-0.938194,-0.838177,-0.453413,0,0.235702,0.916689
3,KT-0001685,0.482284,0.250439,-0.514664,1.794744,1.794744,0.092685,1.236181,0.308173,-0.560154,...,-0.766791,-0.129833,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.397809
4,KT-0001686,0.642138,-2.476299,-0.667554,-1.376570,-1.376570,-0.139945,-0.572946,-0.646185,-1.698210,...,-1.386750,-0.358406,-0.342997,-0.190693,-0.938194,-0.838177,-0.453413,0,0.235702,0.916689


In [33]:
## get descriptors (X)
X = dataTable_merged.iloc[:, 2:]
y = dataTable_merged.iloc[:, 1:2]
print(X.shape, y.shape)

(38, 2223) (38, 1)


##### remove descriptors with too many missing data

In [34]:
def missingValueFilter(desc_all, json_file_imput_param, nan_cutoff=0.2):
    nan_ratio_dict, desc_sele = {}, []
    ## load imputation param file
    import os, json
    if os.path.exists(json_file_imput_param):
        with open(json_file_imput_param, 'r') as ifh:
            dict_imput_param = json.load(ifh)
        ##
        for desc in dict_imput_param:
            if desc in desc_all:
                count_nan = dict_imput_param[desc]['count_nan']
                count_all = dict_imput_param[desc]['count_all']
                nan_ratio = count_nan/count_all
                if desc not in nan_ratio_dict:
                    nan_ratio_dict[desc] = {}
                    nan_ratio_dict[desc]['Descriptor'] = desc
                    nan_ratio_dict[desc]['nan_ratio'] = nan_ratio
                if nan_ratio <= nan_cutoff:
                    nan_ratio_dict[desc]['Select'] = 'Yes'
                    desc_sele.append(desc)
                else:
                    nan_ratio_dict[desc]['Select'] = 'No'
        ## print results
        print(f"\tIn total <{len(nan_ratio_dict)}> desc, there are <{len(desc_sele)}> selected, cutoff is {nan_cutoff}.")
    else:
        print(f"Error! The imputation param file {json_file_imput_param} does not exist")

    nan_ratio_table = pd.DataFrame.from_dict(nan_ratio_dict).T
    return nan_ratio_table, desc_sele

if MissingValueFilter:
    desc_all = list(X.columns)
    json_file_imput_param = f"{desc_folder}/descriptor_imputation_params.dict"
    nan_cutoff = 0.2
    nan_ratio_table, desc_sele = missingValueFilter(desc_all, json_file_imput_param, nan_cutoff)

nan_ratio_table

	In total <2223> desc, there are <2222> selected, cutoff is 0.2.


,Descriptor,nan_ratio,Select
rd_MaxAbsEStateIndex,rd_MaxAbsEStateIndex,0.0,Yes
rd_MaxEStateIndex,rd_MaxEStateIndex,0.0,Yes
rd_MinAbsEStateIndex,rd_MinAbsEStateIndex,0.0,Yes
rd_MinEStateIndex,rd_MinEStateIndex,0.0,Yes
rd_qed,rd_qed,0.0,Yes
...,...,...,...
cx_topology-analyser_largestRingSize,cx_topology-analyser_largestRingSize,0.0,Yes
cx_topology-analyser_largestRingSystemSize,cx_topology-analyser_largestRingSystemSize,0.0,Yes
cx_topology-analyser_rotatableBondCount,cx_topology-analyser_rotatableBondCount,0.0,Yes
custDesc_ADME MDCK(WT) Permeability;Mean;A to B Recovery (%),custDesc_ADME MDCK(WT) Permeability;Mean;A to ...,0.0,Yes


In [35]:
X = X[desc_sele]
print(X.shape)

(38, 2222)


##### Variance-based Filter

In [36]:
def VarianceFilter(X, threshold=0):
    ## Removing features with no variance
    from sklearn.feature_selection import VarianceThreshold
    variance_filter = VarianceThreshold(threshold=threshold)
    variance_filter.fit_transform(X)

    ## generate variance table
    desc_list = list(X.columns)
    variance_list = variance_filter.variances_.tolist()
    variance_dict, desc_sele = {}, []
    for i in range(len(desc_list)):
        variance_dict[i] = {}
        variance_dict[i]['Descriptor'] = desc_list[i]
        variance_dict[i]['Variance'] = variance_list[i]
        if variance_list[i] > threshold:
            variance_dict[i]['Select'] = 'Yes'
            desc_sele.append(desc_list[i])
        else:
            variance_dict[i]['Select'] = 'No'
    variance_table = pd.DataFrame.from_dict(variance_dict).T
    ## print results
    print(f"\tIn total <{len(desc_list)}> desc, there are <{len(desc_sele)}> selected, cutoff is {threshold}.")
    return variance_table, desc_sele

##
if VarianceFilter:
    variance_table, desc_sele = VarianceFilter(X=X, threshold=0.0001)
variance_table

	In total <2222> desc, there are <937> selected, cutoff is 0.0001.


,Descriptor,Variance,Select
0,rd_MaxAbsEStateIndex,1.0,Yes
1,rd_MaxEStateIndex,1.0,Yes
2,rd_MinAbsEStateIndex,1.0,Yes
3,rd_MinEStateIndex,1.0,Yes
4,rd_qed,1.0,Yes
...,...,...,...
2217,cx_topology-analyser_largestRingSize,0.0,No
2218,cx_topology-analyser_largestRingSystemSize,1.0,Yes
2219,cx_topology-analyser_rotatableBondCount,1.0,Yes
2220,custDesc_ADME MDCK(WT) Permeability;Mean;A to ...,1.0,Yes


In [37]:
X = X[desc_sele]
print(X.shape)

(38, 937)


In [38]:
def L2_based_selection(X, y, model_type='regression', penalty_param=0.01):
    ## define estimator model
    if model_type == 'regression':
        from sklearn import linear_model
        estimator = linear_model.Lasso(alpha=penalty_param)
    elif model_type == 'classification':
        from sklearn.svm import LinearSVC
        estimator = LinearSVC(penalty="l2", loss="squared_hinge", dual=True)
    
    ## fit estimator model
    estimator.fit(X, y)
    scores = estimator.coef_

    ## selector model
    from sklearn.feature_selection import SelectFromModel
    selector = SelectFromModel(estimator=estimator, prefit=True)
    select_feature_mask = selector.get_support()

    ## result table
    desc_list = list(X.columns)
    coef_dict, desc_sele = {}, []
    if len(scores) == len(desc_list):
        for i in range(len(desc_list)):
            coef_dict[i] = {}
            coef_dict[i]['Descriptor'] = desc_list[i]
            coef_dict[i]['l2_coef'] = scores[i]
            if select_feature_mask[i]:
                coef_dict[i]['Select'] = 'Yes'
                desc_sele.append(desc_list[i])
            else:
                coef_dict[i]['Select'] = 'No'
    else:
        print(f"Error! The desc (N={len(desc_list)}) does not match the coef scores (N={len(scores)})")

    ## print results
    print(f"\tIn total <{len(desc_list)}> desc, there are <{len(desc_sele)}> selected.")
    coef_table = pd.DataFrame.from_dict(coef_dict).T
    return coef_table, desc_sele

if L2Filter:
    coef_table, desc_sele = L2_based_selection(X=X, y=y, model_type='regression')
coef_table

	In total <937> desc, there are <33> selected.


,Descriptor,l2_coef,Select
0,rd_MaxAbsEStateIndex,-0.446669,Yes
1,rd_MaxEStateIndex,-0.0,No
2,rd_MinAbsEStateIndex,-0.309304,Yes
3,rd_MinEStateIndex,-0.0,No
4,rd_qed,0.0,No
...,...,...,...
932,cx_topology-analyser_heteroAromaticRingCount,0.0,No
933,cx_topology-analyser_largestRingSystemSize,-0.07355,Yes
934,cx_topology-analyser_rotatableBondCount,-0.0,No
935,custDesc_ADME MDCK(WT) Permeability;Mean;A to ...,0.760321,Yes


In [46]:
def RF_based_selection(X, y, model_type='regression', penalty_param=0.01):
    ## define estimator model
    if model_type == 'regression':
        from sklearn.ensemble import RandomForestRegressor
        estimator = RandomForestRegressor()
    elif model_type == 'classification':
        from sklearn.ensemble import RandomForestClassifier
        estimator = RandomForestClassifier()
    
    ## fit estimator model
    estimator.fit(X, y)
    scores = estimator.feature_importances_

    ## selector model
    from sklearn.feature_selection import SelectFromModel
    selector = SelectFromModel(estimator=estimator, prefit=True)
    select_feature_mask = selector.get_support()

    ## result table
    desc_list = list(X.columns)
    FI_dict, desc_sele = {}, []
    if len(scores) == len(desc_list):
        for i in range(len(desc_list)):
            FI_dict[i] = {}
            FI_dict[i]['Descriptor'] = desc_list[i]
            FI_dict[i]['feature_importance'] = scores[i]
            if select_feature_mask[i]:
                FI_dict[i]['Select'] = 'Yes'
                desc_sele.append(desc_list[i])
            else:
                FI_dict[i]['Select'] = 'No'
    else:
        print(f"Error! The desc (N={len(desc_list)}) does not match the feature importance (N={len(scores)})")

    ## print results
    print(f"\tIn total <{len(desc_list)}> desc, there are <{len(desc_sele)}> selected.")
    FI_table = pd.DataFrame.from_dict(FI_dict).T
    return FI_table, desc_sele

if FeatureImportanceFilter:
    FI_table, desc_sele = RF_based_selection(X=X, y=ynp, model_type='regression')
FI_table


	In total <937> desc, there are <89> selected.


,Descriptor,feature_importance,Select
0,rd_MaxAbsEStateIndex,0.000302,No
1,rd_MaxEStateIndex,0.000842,No
2,rd_MinAbsEStateIndex,0.00245,Yes
3,rd_MinEStateIndex,0.036428,Yes
4,rd_qed,0.019886,Yes
...,...,...,...
932,cx_topology-analyser_heteroAromaticRingCount,0.0,No
933,cx_topology-analyser_largestRingSystemSize,0.0,No
934,cx_topology-analyser_rotatableBondCount,0.002032,Yes
935,custDesc_ADME MDCK(WT) Permeability;Mean;A to ...,0.04749,Yes


In [41]:
y

,ADME MDCK(WT) Permeability;Mean;A to B Papp (10^-6 cm/s);(Num)
0,0.298013
1,0.767547
2,0.186651
3,0.482284
4,0.642138
5,15.407308
6,0.426379
7,0.328750
8,0.301589
9,2.207218


In [45]:
ynp = y.to_numpy().reshape((len(y),))

In [24]:
score_table_dict = {}
score_table_dict['MissingValueFilter'] =nan_ratio_table
score_table_dict['VarianceFilter'] = variance_table
score_table_dict['L2Filter'] = coef_table
score_table_dict['FIFilter'] = FI_table

In [25]:
folderPathOut = './results'
score_table_merged = pd.DataFrame(columns=['Descriptor'])
for filterType in score_table_dict:
    this_Table = score_table_dict[filterType]
    this_Table.to_csv(f"{folderPathOut}/feature_scoring_{filterType}.csv", index=False)
    
    this_Table = this_Table.rename(columns={'Select': f'select_{filterType}'})
    score_table_merged = score_table_merged.merge(right=this_Table, on='Descriptor', how='outer')
score_table_merged.to_csv(f"{folderPathOut}/feature_scoring_merged.csv", index=False)


In [26]:
score_table_merged

,Descriptor,nan_ratio,select_MissingValueFilter,Variance,select_VarianceFilter,l2_coef,select_L2Filter,feature_importance,select_FIFilter
0,custDesc_ADME MDCK(WT) Permeability;Mean;A to ...,0.0,Yes,1.0,Yes,0.760321,Yes,0.020314,Yes
1,custDesc_ADME MDCK(WT) Permeability;Mean;B to ...,0.0,Yes,1.0,Yes,1.484672,Yes,0.062362,Yes
2,cx_charge_formalCharge,0.0,Yes,1.0,Yes,0.0,No,0.000002,No
3,cx_elemental-analysis_mass,0.0,Yes,1.0,Yes,0.0,No,0.008807,Yes
4,cx_hbda_acceptorAtomCount,0.0,Yes,1.0,Yes,-0.0,No,0.000007,No
...,...,...,...,...,...,...,...,...,...
2218,rd_fr_thiocyan,0.0,Yes,0.0,No,NaN,NaN,NaN,NaN
2219,rd_fr_thiophene,0.0,Yes,0.0,No,NaN,NaN,NaN,NaN
2220,rd_fr_unbrch_alkane,0.0,Yes,1.0,Yes,-0.150588,Yes,0.0,No
2221,rd_fr_urea,0.0,Yes,0.0,No,NaN,NaN,NaN,NaN


In [27]:
dataTable_merged

,Compound Name,ADME MDCK(WT) Permeability;Mean;A to B Papp (10^-6 cm/s);(Num),custDesc_ADME MDCK(WT) Permeability;Mean;A to B Recovery (%),custDesc_ADME MDCK(WT) Permeability;Mean;B to A Papp (10^-6 cm/s);(Num),rd_MaxAbsEStateIndex,rd_MaxEStateIndex,rd_MinAbsEStateIndex,rd_MinEStateIndex,rd_qed,rd_SPS,...,cx_topology-analyser_chiralCenterCount,cx_topology-analyser_fsp3,cx_topology-analyser_fusedAliphaticRingCount,cx_topology-analyser_fusedAromaticRingCount,cx_topology-analyser_fusedRingCount,cx_topology-analyser_heteroRingCount,cx_topology-analyser_heteroAromaticRingCount,cx_topology-analyser_largestRingSize,cx_topology-analyser_largestRingSystemSize,cx_topology-analyser_rotatableBondCount
0,KT-0001674,0.298013,-0.012508,-0.449259,1.820147,1.820147,0.753577,1.248948,0.347800,-0.560154,...,-0.766791,0.171082,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.397809
1,KT-0001675,0.767547,0.034798,1.291708,1.808079,1.808079,2.133891,1.271090,0.299537,-0.447405,...,-0.766791,0.737510,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.178726
2,KT-0001676,0.186651,-2.419224,-0.765732,-1.339989,-1.339989,-0.372173,-0.569409,-0.658017,-1.698210,...,-1.386750,-0.142262,-0.342997,-0.190693,-0.938194,-0.838177,-0.453413,0,0.235702,0.916689
3,KT-0001685,0.482284,0.250439,-0.514664,1.794744,1.794744,0.092685,1.236181,0.308173,-0.560154,...,-0.766791,-0.129833,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.397809
4,KT-0001686,0.642138,-2.476299,-0.667554,-1.376570,-1.376570,-0.139945,-0.572946,-0.646185,-1.698210,...,-1.386750,-0.358406,-0.342997,-0.190693,-0.938194,-0.838177,-0.453413,0,0.235702,0.916689
5,KT-0001844,15.407308,2.076327,2.568832,1.060774,1.060774,1.701207,1.364561,4.045526,-0.292028,...,-1.386750,0.454296,-0.342997,-2.002271,-2.309401,-2.471547,-2.367826,0,-4.242641,-1.931389
6,KT-0001862,0.426379,-0.231540,-0.012761,1.782676,1.782676,1.901663,1.258324,0.273973,-0.447405,...,-0.766791,0.454296,-0.342997,-0.190693,-0.938194,-0.838177,-1.410620,0,0.235702,-0.178726
7,KT-0001038,0.328750,1.127342,0.131559,0.605769,0.605769,-1.216474,-0.229725,-0.820891,-1.224201,...,-0.146832,-1.343025,-0.342997,-2.002271,-0.938194,0.795193,0.503793,0,0.235702,2.888436
8,KT-0001876,0.301589,1.150675,0.082074,0.449726,0.449726,0.426881,-0.208634,-0.502662,-0.660876,...,-0.146832,-2.390160,-0.342997,-2.002271,-2.309401,0.795193,0.503793,0,-4.242641,0.697606
9,KT-0001879,2.207218,-0.308007,0.372136,0.508178,0.508178,-0.961866,-0.161566,-0.598368,-0.879323,...,-0.146832,0.454296,-0.342997,-0.190693,-0.938194,-0.021492,1.460999,0,0.235702,1.135772


In [29]:
dataTable_merged.columns[0]

'Compound Name'